In [3]:
#firsr of all import all package and load the file
import string
from math import log
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict, Counter
from nltk.corpus import stopwords    
from nltk.tag import StanfordNERTagger

from nltk.tokenize import word_tokenize
import json
import nltk
nltk.download('averaged_perceptron_tagger')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
jar = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/stanford-ner.jar'
model = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/classifiers/english.conll.4class.distsim.crf.ser.gz'
model1 = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz'
model2 = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-ner-2018-02-27/classifiers/english.muc.7class.distsim.crf.ser.gz'

punc = string.punctuation
stopwordsPart = set(stopwords.words('english'))
stopwordsPart.remove('the')  
stopwordsPart.remove('of') 
stopwordsAll = set(stopwords.words('english'))
def opne_json(text):
    with open(text,'r') as input_file:
        document = json.load(input_file)
    return document



def get_tag_model(model,jar):
    return StanfordNERTagger(model,jar)


person_model = get_tag_model(model,jar)
person_model2 = get_tag_model(model1,jar)
number_model = get_tag_model(model2,jar)




documents_dict = opne_json("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/documents.json")
test_dict = opne_json("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/testing.json")
dev_dict = opne_json("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/devel.json")
train_dict = opne_json("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/training.json")
#query_lables = opne_json("QuestionLabel.json")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shimei/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [4]:
#get paragraph from the training data
def get_paragraph(docid,documents_dict):
    #get the paragraph that contains the answer
    for i in documents_dict:
        if i['docid'] == docid:
            document = i['text']
            break
    return document



In [5]:
#get TF 
def term_freqs(document):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for sentence in document:
        for token in word_tokenize(sentence):
            if token not in stopwordsAll and token not in punc:  
                term = lemmatizer.lemmatize(token.lower())
                tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1 
                tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1 
        doc_id += 1
    return tfs,doc_id+1,tfs_forward




In [6]:
#build TF_IDF model
def get_tfidf(tfs, total_docment,tfs_forward):
    document_length = {}
    for doc_id,doc_list in tfs_forward.items():
        length = 0
        for term, freq in doc_list.items():
            length += freq ** 2
        length = length **0.5
        document_length[doc_id] =  length
    tfidf = defaultdict(dict)
    for term, doc_list in tfs.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items(): 
            tfidf[term][doc_id] = (float(tfs[term][doc_id]) * log(total_docment / df))# / document_length[doc_id]
    return tfidf



In [7]:
#find top_k paragraph that may contain the answer
def get_top_k_document(tfidf,query,k,document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        if token not in stopwordsAll:  
             term = lemmatizer.lemmatize(token.lower())
             term_tfidf = tfidf[term]
             for docid, weight in term_tfidf.items():
                 top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    top_document = []
    for document_id,weight in top_document_id:
        top_document.append(document[document_id])
    return top_document


In [8]:
#filter the key words in query
def get_open_class_word(query):
    tagged = nltk.pos_tag(word_tokenize(query), tagset="universal")
    return [p[0] for p in tagged if (p[1] in ["NOUN","VERB"] or p[0].isdigit()) and p[0] not in ["did","do","was","were"]]

In [9]:
#combine the NER with same tag
def same_tag(ner_output):
    word,tag = '',''
    combo = []
    for word1,tag1 in ner_output:
        if tag1 == tag:
            if word[-1] in ['(',')']:
                word += word1
            if word1 in [')']:
                 word += word1
            else:     
                word += " " + word1
        else:
            combo.append((word,tag))
            tag = tag1
            word = word1
            continue
    if len(combo) != 0:
        combo.pop(0)
    return combo

In [15]:
def most_in(key_words,sentence):
    all_in = True
    len1 = len(key_words)
    word_in = 0
    for i in key_words:
        try:
            index = sentence.index(i)
            word_in += 1 
        except ValueError:
            continue
    return len1 < 2*word_in

def in_key_words(word,key_words):
    in_key = False
    for i in key_words:
        if word.find(i) != -1:       
            in_key = True
            break
    return in_key
            

In [16]:
document = get_paragraph(409,documents_dict)
tfs,total_docment,tfs_forward = term_freqs(document)
tfidf = get_tfidf(tfs, total_docment,tfs_forward)


In [17]:
query ="Who performance in Anne of the Thousand Days garnered a Best Actor nod?"
top_k = get_top_k_document(tfidf,query,5,document)


In [19]:

rules = {
            'name':'PERSON',
            'country': 'LOCATION',
            'capital': 'LOCATION',
            'newspaper':'ORGANIZATION',
            'company':'ORGANIZATION',
            'organization':'ORGANIZATION',
            'person':'PERSON',
            'cents':'MONEY',
            'dollar':'MONEY',
            'river':'LOCATION',
            'location': 'LOCATION',
            'mountain':'LOCATION',
            'website':'ORGANIZATION',
            'airline':'ORGANIZATION',
            'percentage':'PERCENT',
            'population':'NUMBER',
            'decade':'DATE',
            'where': 'LOCATION',
            'when': 'DATE',           
            'who': 'PERSON',     
            'what scientist':'PERSON',
            'what time':'TIME',
            'what athlete':'PERSON',
            'which athlete':'PERSON',
            'what people': 'PERSON',
            'what date':'DATE',
            'what day':'DATE',
            'what month':'DATE',
            'what year': 'DATE',
            'what era':'DATE',
            'What age':'NUMBER',
            'how old':'NUMBER',
            'which anniversary':'DATE',
            'what century':'DATE',
            'time period':'NUMBER',
            'what period':'NUMBER',
            'what city' : 'LOCATION',
            'which city' : 'LOCATION',
            'what county':'LOCATION',
            'what street':'LOCATION',
            'what museum':'LOCATION',
            'what region':'LOCATION',
            'what road':'LOCATION',
            'which publication':'ORGANIZATION',
            'which council':'ORGANIZATION',
            'what government':'ORGANIZATION',
            'what college':'ORGANIZATION',
            'which supporters' : 'PERSON',
            'which footballer': 'PERSON',
            'which actor':'PERSON',
            'Which actress':'PERSON',
            'which American actress':'PERSON',
            'what activists':'PERSON',
            'which team member' : 'PERSON',
            'what football star': 'PERSON',
            'which blogger': 'PERSON',
            'which torchbearer':'PERSON',
            'which wheelchair-bound torchbearer' : 'PERSON',
            'how much': 'NUMBER',
            'how many': 'NUMBER',
            'how far':'NUMBER',
            'how long':'NUMBER',
            'how much of': 'PERCENT',
            'How many of': 'PERCENT',
            'by how much': 'PERCENT'        
}

money_list = ['cost', 'worth', 'spend', 'money', 'worth', 'invest']

def tag_answer_type(question):
    answer_type = 'O'
    processed_question = []
    processed_question_str = None
    for token in [question]:
        processed_question.append(token.lower())
    processed_question_str = " ".join(x for x in processed_question)
    for k,v in rules.items():
        if k in processed_question_str:
            #print(k)
            if k == 'how much':
                for item in money_list:
                    #print("item", item)
                    if item in processed_question_str:
                        answer_type = 'MONEY'
                    else:
                        continue
            else:
                answer_type = rules.get(k, "O")    
    return answer_type

In [20]:
def get_answer_list(query,top_k):
    key_words = get_open_class_word(query)

    answer_list = {}
    answer_type = tag_answer_type(query)
    if answer_type == "O":
        return None
    
    for ans_sentence in top_k:
        if most_in(key_words,ans_sentence) == False:
            continue
        word_list =  []
        for word in word_tokenize(ans_sentence):
            word_list.append(word)    
        word_list_tag = number_model.tag(word_list)
        word_list_tag = same_tag(word_list_tag)
        for word,tags in word_list_tag:
            if word not in answer_list.keys():
                if word not in stopwordsAll and word not in punc and tags == answer_type and word not in key_words and in_key_words(word,key_words) == False:
                    distance_list = []
                    distance = 0
                    for key_word in key_words:
                        try:
                            index = ans_sentence.index(key_word)
                            distance_list.append(index)
                        except ValueError:
                            distance_list.append(5000)
                    for index in distance_list:
                        try:
                            distance += abs(index - ans_sentence.index(word))
                        except ValueError:
                            print ('')
                    answer_list[word] = distance
    if  len(answer_list.items()) != 0:
        return sorted(answer_list.items(), key=operator.itemgetter(1), reverse = True)[0][0].lower()
        return None

In [21]:
import operator
print (get_answer_list(query,top_k))

hal b. wallis


In [22]:
import csv
def get_accuracy(train_dict,documents_dict):
    docidstart = train_dict[0]['docid']
    document = get_paragraph(docidstart,documents_dict)
    tfs,total_docment,tfs_forward = term_freqs(document)
    tfidf = get_tfidf(tfs, total_docment,tfs_forward)
    for i in train_dict:
        docid = i['docid']
        if docid != docidstart:
            document = get_paragraph(docid,documents_dict)
            tfs,total_docment,tfs_forward = term_freqs(document)
            tfidf = get_tfidf(tfs, total_docment,tfs_forward)
            docidstart = docid
        query = i['question']
        qaid = i['id']
        top_k = get_top_k_document(tfidf,query,1,document)
        potiential_answer = get_answer_list(query,top_k)
        
def output(test_dict,documents_dict):
    output = open("out.txt", "w")
    docidstart = train_dict[0]['docid']
    document = get_paragraph(docidstart,documents_dict)
    tfs,total_docment,tfs_forward = term_freqs(document)
    tfidf = get_tfidf(tfs, total_docment,tfs_forward)
    for i in test_dict:
        docid = i['docid']
        if docid != docidstart:
            document = get_paragraph(docid,documents_dict)
            tfs,total_docment,tfs_forward = term_freqs(document)
            tfidf = get_tfidf(tfs, total_docment,tfs_forward)
            docidstart = docid
        query = i['question']
        qaid = i['id']
        top_k = get_top_k_document(tfidf,query,1,document)
        potiential_answer = get_answer_list(query,top_k)
        print(potiential_answer)
        print("qaid",qaid)
        print("potiential_answer",potiential_answer)
        output.write(str(qaid) + "," + str(potiential_answer) + '\n')
    
#output(test_dict,documents_dict)

In [23]:
from gensim.models import Word2Vec
import json
from nltk.tokenize import sent_tokenize, word_tokenize

modelname = "/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/mymodel"

#train the model
corpus = []
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/documents.json",'r') as docs:
    data = json.load(docs)
    for i in range(len(data)):
        for para in data[i]['text']:
            sents = sent_tokenize(para)
            for sent in sents:
                tokens = word_tokenize(sent)
                corpus.append(tokens)
    
model = Word2Vec(corpus,min_count=1,hs=1,negative=0)
model.save(modelname)

#model = Word2Vec.load(modelname)
#model.wv['computer']

In [42]:
# word2vec for what type question
import json
import re
from nltk.tag import StanfordPOSTagger
jar1 = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-postagger-2018-02-27/stanford-postagger.jar'
model3 = '/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/stanford-postagger-2018-02-27/models/english-bidirectional-distsim.tagger'
all_questions_dict = {}
all_docids = {}
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/testing.json",'r') as test_file:
    test_data = json.load(test_file)
    for i in range(len(test_data)):
        question = test_data[i]['question']
        docid = test_data[i]['docid']
        all_questions_dict[i] = question
        all_docids[i] = docid
print(len(all_questions_dict))
 
def get_POS_tag_model(model,jar):
    return StanfordPOSTagger(model,jar)

what_model = get_POS_tag_model(model3, jar1)

possible_tag_list = ["JJ","JJR","JJS","NN","NNS","NNP","NNPS"]

output = open("final.txt", "w")
with open('out.txt', 'rt') as f:
    for line in f:
        #print("line",line)
        item = line.split(',')
        #print(item)
        if item[1].startswith("None"):
            current_question = all_questions_dict.get(int(item[0]), 0)
            print("current_question",current_question)
            document = get_paragraph(all_docids.get(int(item[0]), 0),documents_dict)
            tfs,total_docment,tfs_forward = term_freqs(document)
            tfidf = get_tfidf(tfs, total_docment,tfs_forward)
            top_k = get_top_k_document(tfidf,current_question,1,document)
            #print(top_k)
            POS_top_k = what_model.tag_sents([[" ".join(x) for x in [word_tokenize(top_k[0])]]])
            #print("POS_top_k",POS_top_k)
            poossible_answers = []
            for tag in possible_tag_list:
                for word_items in POS_top_k:
                    for word_item in word_items:
                        #print("word_item",word_item)
                        if word_item[1] == tag and word_item[0] != '(' and word_item[0] != ')' and re.match(r'[a-zA-Z0-9]+', word_item[0]):
                            poossible_answers.append(word_item[0])
            #print("poossible_answers",poossible_answers)
            
            processed_question = []
            que_list = word_tokenize(current_question)
            for i in range(len(que_list)-1):
                processed_question.append(que_list[i].lower())
            #print("processed_question",processed_question)
            #print(len(processed_question)-1)
            for i in range(len(processed_question)):
                #print("processed_question[i]",processed_question[i])
                if processed_question[i] == "what":
                    score_dict = {}
                    for possible_answer in poossible_answers:
                        new_list = []
                        processed_question[i] = possible_answer
                        new_list = [" ".join(x) for x in [processed_question]]
                        #print("new_list",new_list)
                        score = model.score(new_list)
                        score_dict[possible_answer] = score[0]
                    #print(score_dict)
                    answer = sorted(score_dict.items(), key=operator.itemgetter(1), reverse = True)[0][0].lower()
                    print("answer",answer)
                    output.write(item[0] + "," + str(answer) + '\n')
        else:
            output.write(line)
    


    
    

3618
current_question Modern browser support standards-based and defacto what?


/Users/shimei/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


answer web
current_question What do people typically call a web browser?
answer web
current_question What is it called when content is changed from markup to an interactive document?
answer xml
current_question What platform is a browser used on?
answer ie
current_question When was Firefox released?
current_question Who released the Internet Explorer browser?
current_question When was the first browser created?
current_question HTTP Secure is supported by what?
answer uri
current_question The Commission felt that the bundling undermined what?
answer web
current_question The release of Internet Explorer started the first what?
answer may
current_question What does IRC stand for?
answer irc
current_question Who investigated the bundling of the IE browser with Windows OS?
current_question Which browser is the newest to enter the field?
current_question What does ftp stand for?
answer uri
current_question What does http stand for?
answer uri
current_question Features on a browser can range

KeyboardInterrupt: 